In [1]:
#!nvidia-smi
#using a GeForce GTX1080 Ti for reproducibility for all timing experiments

In [2]:
import torch
import torchvision
from torch import nn, optim, autograd
from torch.nn import functional as F
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.autograd import Variable
import numpy as np
from sklearn.utils import shuffle as skshuffle
from math import *
from backpack import backpack, extend
from backpack.extensions import KFAC, DiagHessian, DiagGGNMC
from sklearn.metrics import roc_auc_score
import scipy
from tqdm import tqdm, trange
import pytest
from DirLPA_utils import * 
import time
from torch.distributions.normal import Normal

import matplotlib.pyplot as plt

s = 123
np.random.seed(s)
torch.manual_seed(s)
torch.cuda.manual_seed(s)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
def LPADirNN(num_classes=10, num_LL=256):
    
    features = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, 5),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2,2),
        torch.nn.Conv2d(32, 64, 5),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2,2),
        torch.nn.Flatten(),
        torch.nn.Linear(4 * 4 * 64, num_LL), #changed from 500
        torch.nn.Linear(num_LL, num_classes)  #changed from 500
    )
    return(features)

In [4]:
BATCH_SIZE_TRAIN_MNIST = 32#128
BATCH_SIZE_TEST_MNIST = 32#128
MAX_ITER_MNIST = 6
LR_TRAIN_MNIST = 10e-6

In [5]:
MNIST_transform = torchvision.transforms.ToTensor()

MNIST_train = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=True,
        download=True,
        transform=MNIST_transform)

mnist_train_loader = torch.utils.data.dataloader.DataLoader(
    MNIST_train,
    batch_size=BATCH_SIZE_TRAIN_MNIST,
    shuffle=True
)


MNIST_test = torchvision.datasets.MNIST(
        '~/data/mnist',
        train=False,
        download=True,
        transform=MNIST_transform)

mnist_test_loader = torch.utils.data.dataloader.DataLoader(
    MNIST_test,
    batch_size=BATCH_SIZE_TEST_MNIST,
    shuffle=False,
)

In [6]:
mnist_model = LPADirNN(num_LL=256).cuda()
loss_function = torch.nn.CrossEntropyLoss()

mnist_train_optimizer = torch.optim.Adam(mnist_model.parameters(), lr=1e-3, weight_decay=5e-4)
MNIST_PATH = "pretrained_weights/MNIST_pretrained_10_classes.pth"

In [7]:
#Training routine

def train(model, train_loader, optimizer, max_iter, path, verbose=True):
    max_len = len(train_loader)

    for iter in range(max_iter):
        for batch_idx, (x, y) in enumerate(train_loader):
            
            x, y = x.cuda(), y.cuda()
            
            output = model(x)

            accuracy = get_accuracy(output, y)

            loss = loss_function(output, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if verbose and batch_idx % 50 == 0:
                print(
                    "Iteration {}; {}/{} \t".format(iter, batch_idx, max_len) +
                    "Minibatch Loss %.3f  " % (loss) +
                    "Accuracy %.0f" % (accuracy * 100) + "%"
                )

    print("saving model at: {}".format(path))
    torch.save(mnist_model.state_dict(), path)

In [8]:
#train(mnist_model, mnist_train_loader, mnist_train_optimizer, MAX_ITER_MNIST, MNIST_PATH, verbose=True)

In [9]:
#predict in distribution
MNIST_PATH = "pretrained_weights/MNIST_pretrained_10_classes.pth"

mnist_model = LPADirNN(num_LL=256).cuda()
print("loading model from: {}".format(MNIST_PATH))
mnist_model.load_state_dict(torch.load(MNIST_PATH))
mnist_model.eval()

acc = []

max_len = len(mnist_test_loader)
for batch_idx, (x, y) in enumerate(mnist_test_loader):

    x, y = x.cuda(), y.cuda()
    output = mnist_model(x)

    accuracy = get_accuracy(output, y)
    if batch_idx % 10 == 0:
        print(
            "Batch {}/{} \t".format(batch_idx, max_len) + 
            "Accuracy %.0f" % (accuracy * 100) + "%"
        )
    acc.append(accuracy)

avg_acc = np.mean(acc)
print('overall test accuracy on MNIST: {:.02f} %'.format(avg_acc * 100))


loading model from: pretrained_weights/MNIST_pretrained_10_classes.pth
Batch 0/313 	Accuracy 100%
Batch 10/313 	Accuracy 97%
Batch 20/313 	Accuracy 94%
Batch 30/313 	Accuracy 97%
Batch 40/313 	Accuracy 97%
Batch 50/313 	Accuracy 97%
Batch 60/313 	Accuracy 100%
Batch 70/313 	Accuracy 100%
Batch 80/313 	Accuracy 100%
Batch 90/313 	Accuracy 97%
Batch 100/313 	Accuracy 100%
Batch 110/313 	Accuracy 94%
Batch 120/313 	Accuracy 97%
Batch 130/313 	Accuracy 97%
Batch 140/313 	Accuracy 100%
Batch 150/313 	Accuracy 97%
Batch 160/313 	Accuracy 100%
Batch 170/313 	Accuracy 100%
Batch 180/313 	Accuracy 100%
Batch 190/313 	Accuracy 97%
Batch 200/313 	Accuracy 100%
Batch 210/313 	Accuracy 100%
Batch 220/313 	Accuracy 100%
Batch 230/313 	Accuracy 100%
Batch 240/313 	Accuracy 100%
Batch 250/313 	Accuracy 100%
Batch 260/313 	Accuracy 100%
Batch 270/313 	Accuracy 100%
Batch 280/313 	Accuracy 100%
Batch 290/313 	Accuracy 100%
Batch 300/313 	Accuracy 100%
Batch 310/313 	Accuracy 100%
overall test accuracy o

In [10]:
# get all weight dists
def get_Hessian_NN(model, train_loader, prec0, device='cpu', verbose=True):
    lossfunc = torch.nn.CrossEntropyLoss()

    extend(lossfunc, debug=False)
    extend(model, debug=False)

    Hessian_diag = []
    for param in model.parameters():
        ps = param.size()
        print("parameter size: ", ps)
        Hessian_diag.append(torch.zeros(ps, device=device))

    var0 = 1/prec0
    max_len = len(train_loader)

    with backpack(DiagHessian()):

        for batch_idx, (x, y) in enumerate(train_loader):

            if device == 'cuda':
                x, y = x.float().cuda(), y.long().cuda()

            model.zero_grad()
            lossfunc(model(x), y).backward()

            with torch.no_grad():
                # Hessian of weight
                for idx, param in enumerate(model.parameters()):

                    H_ = param.diag_h
                    #add prior here
                    H_ += var0 * torch.ones(H_.size(), device=device)

                    rho = 1-1/(batch_idx+1)

                    Hessian_diag[idx] = rho*Hessian_diag[idx] + (1-rho)* H_
            
            if verbose and batch_idx%20==0:
                print("Batch: {}/{}".format(batch_idx, max_len))
    
    return(Hessian_diag)

In [11]:
# sample from weights
def get_param_dists(model, Hessian):
    param_dists = []
    for idx, param in enumerate(model.parameters()):
        ps = param.size()
        mu = param.view(-1)
        Sigma = Hessian[idx].view(-1)
        print("mu size: ", mu.size())
        print("Sigma size: ", Sigma.size())
        dist = Normal(mu, Sigma)
        param_dists.append(dist)
        
    return(param_dists)

## Load additional data

In [12]:
BATCH_SIZE_TEST_FMNIST = 32#128
BATCH_SIZE_TEST_KMNIST = 32#128

In [13]:
FMNIST_test = torchvision.datasets.FashionMNIST(
        '~/data/fmnist', train=False, download=True,
        transform=MNIST_transform)   #torchvision.transforms.ToTensor())

FMNIST_test_loader = torch.utils.data.DataLoader(
    FMNIST_test,
    batch_size=BATCH_SIZE_TEST_FMNIST, shuffle=False)

In [14]:
KMNIST_test = torchvision.datasets.KMNIST(
        '~/data/kmnist', train=False, download=True,
        transform=MNIST_transform)

KMNIST_test_loader = torch.utils.data.DataLoader(
    KMNIST_test,
    batch_size=BATCH_SIZE_TEST_KMNIST, shuffle=False)

In [15]:
"""Load notMNIST"""

import os
import numpy as np
import torch
from PIL import Image
from torch.utils.data.dataset import Dataset
from matplotlib.pyplot import imread
from torch import Tensor

"""
Loads the train/test set. 
Every image in the dataset is 28x28 pixels and the labels are numbered from 0-9
for A-J respectively.
Set root to point to the Train/Test folders.
"""

# Creating a sub class of torch.utils.data.dataset.Dataset
class notMNIST(Dataset):

    # The init method is called when this class will be instantiated
    def __init__(self, root, transform):
        
        #super(notMNIST, self).__init__(root, transform=transform)

        self.transform = transform
        
        Images, Y = [], []
        folders = os.listdir(root)

        for folder in folders:
            folder_path = os.path.join(root, folder)
            for ims in os.listdir(folder_path):
                try:
                    img_path = os.path.join(folder_path, ims)
                    Images.append(np.array(imread(img_path)))
                    Y.append(ord(folder) - 65)  # Folders are A-J so labels will be 0-9
                except:
                    # Some images in the dataset are damaged
                    print("File {}/{} is broken".format(folder, ims))
        data = [(x, y) for x, y in zip(Images, Y)]
        self.data = data
        self.targets = torch.Tensor(Y)

    # The number of items in the dataset
    def __len__(self):
        return len(self.data)

    # The Dataloader is a generator that repeatedly calls the getitem method.
    # getitem is supposed to return (X, Y) for the specified index.
    def __getitem__(self, index):
        img = self.data[index][0]

        if self.transform is not None:
            img = self.transform(img)
            
        # Input for Conv2D should be Channels x Height x Width
        img_tensor = Tensor(img).view(1, 28, 28).float()
        label = self.data[index][1]
        return (img_tensor, label)

In [16]:
root = os.path.expanduser('~/data')

# Instantiating the notMNIST dataset class we created
notMNIST_test = notMNIST(root=os.path.join(root, 'notMNIST_small'),
                               transform=MNIST_transform)

# Creating a dataloader
not_mnist_test_loader = torch.utils.data.dataloader.DataLoader(
                            dataset=notMNIST_test,
                            batch_size=BATCH_SIZE_TEST_KMNIST,
                            shuffle=False)

File F/Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png is broken
File A/RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png is broken


In [17]:
targets = MNIST_test.targets.numpy()
targets_FMNIST = FMNIST_test.targets.numpy()
targets_notMNIST = notMNIST_test.targets.numpy().astype(int)
targets_KMNIST = KMNIST_test.targets.numpy()

In [18]:
@torch.no_grad()
def predict_MAP(model, test_loader, num_samples=1, cuda=False):
    py = []

    max_len = int(np.ceil(len(test_loader.dataset)/len(test_loader)))
    for batch_idx, (x, y) in enumerate(test_loader):

        if cuda:
            x, y = x.cuda(), y.cuda()

        py_ = 0
        for _ in range(num_samples):
            py_ += torch.softmax(model(x), 1)
        py_ /= num_samples

        py.append(py_)
    return torch.cat(py, dim=0)

# MAP estimate

In [19]:
mnist_test_in_MAP = predict_MAP(mnist_model, mnist_test_loader, cuda=True).cpu().numpy()
mnist_test_out_fmnist_MAP = predict_MAP(mnist_model, FMNIST_test_loader, cuda=True).cpu().numpy()
mnist_test_out_notMNIST_MAP = predict_MAP(mnist_model, not_mnist_test_loader, cuda=True).cpu().numpy()
mnist_test_out_KMNIST_MAP = predict_MAP(mnist_model, KMNIST_test_loader, cuda=True).cpu().numpy()

In [20]:
acc_in_MAP, prob_correct_in_MAP, ent_in_MAP, MMC_in_MAP = get_in_dist_values(mnist_test_in_MAP, targets)
acc_out_FMNIST_MAP, prob_correct_out_FMNIST_MAP, ent_out_FMNIST_MAP, MMC_out_FMNIST_MAP, auroc_out_FMNIST_MAP = get_out_dist_values(mnist_test_in_MAP, mnist_test_out_fmnist_MAP, targets_FMNIST)
acc_out_notMNIST_MAP, prob_correct_out_notMNIST_MAP, ent_out_notMNIST_MAP, MMC_out_notMNIST_MAP, auroc_out_notMNIST_MAP = get_out_dist_values(mnist_test_in_MAP, mnist_test_out_notMNIST_MAP, targets_notMNIST)
acc_out_KMNIST_MAP, prob_correct_out_KMNIST_MAP, ent_out_KMNIST_MAP, MMC_out_KMNIST_MAP, auroc_out_KMNIST_MAP = get_out_dist_values(mnist_test_in_MAP, mnist_test_out_KMNIST_MAP, targets_KMNIST)

In [21]:
print_in_dist_values(acc_in_MAP, prob_correct_in_MAP, ent_in_MAP, MMC_in_MAP, 'mnist', 'MAP')
print_out_dist_values(acc_out_FMNIST_MAP, prob_correct_out_FMNIST_MAP, ent_out_FMNIST_MAP, MMC_out_FMNIST_MAP, auroc_out_FMNIST_MAP, 'FMNIST', 'MAP')
print_out_dist_values(acc_out_notMNIST_MAP, prob_correct_out_notMNIST_MAP, ent_out_notMNIST_MAP, MMC_out_notMNIST_MAP, auroc_out_notMNIST_MAP, 'notMNIST', 'MAP')
print_out_dist_values(acc_out_KMNIST_MAP, prob_correct_out_KMNIST_MAP, ent_out_KMNIST_MAP, MMC_out_KMNIST_MAP, auroc_out_KMNIST_MAP, 'KMNIST', 'MAP')

[In, MAP, mnist] Accuracy: 0.990; average entropy: 0.035;     MMC: 0.989; Prob @ correct: 0.100
[Out-MAP, KFAC, FMNIST] Accuracy: 0.108; Average entropy: 1.390;    MMC: 0.514; AUROC: 0.993; Prob @ correct: 0.100
[Out-MAP, KFAC, notMNIST] Accuracy: 0.108; Average entropy: 0.808;    MMC: 0.709; AUROC: 0.956; Prob @ correct: 0.100
[Out-MAP, KFAC, KMNIST] Accuracy: 0.083; Average entropy: 0.861;    MMC: 0.688; AUROC: 0.973; Prob @ correct: 0.100


In [22]:
import numpy as np
print(np.shape(mnist_test_in_MAP))

(10000, 10)


In [23]:
#MAP estimate
#seeds are 123,124,125,126,127
acc_in = [0.992]
mmc_in = [0.990]
mmc_out_fmnist = [0.552]
mmc_out_notmnist = [0.709]
mmc_out_kmnist = [0.679]

auroc_out_fmnist = [0.991]
auroc_out_notmnist = [0.960]
auroc_out_kmnist = [0.979]

print("accuracy: {:.03f} with std {:.03f}".format(np.mean(acc_in), np.std(acc_in)))

print("MMC in: {:.03f} with std {:.03f}".format(np.mean(mmc_in), np.std(mmc_in)))
print("MMC out fmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_fmnist), np.std(mmc_out_fmnist)))
print("MMC out notmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_notmnist), np.std(mmc_out_notmnist)))
print("MMC out kmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_kmnist), np.std(mmc_out_kmnist)))

print("AUROC out fmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_fmnist), np.std(auroc_out_fmnist)))
print("AUROC out notmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_notmnist), np.std(auroc_out_notmnist)))
print("AUROC out kmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_kmnist), np.std(auroc_out_kmnist)))

accuracy: 0.992 with std 0.000
MMC in: 0.990 with std 0.000
MMC out fmnist: 0.552 with std 0.000
MMC out notmnist: 0.709 with std 0.000
MMC out kmnist: 0.679 with std 0.000
AUROC out fmnist: 0.991 with std 0.000
AUROC out notmnist: 0.960 with std 0.000
AUROC out kmnist: 0.979 with std 0.000


# Diag Hessian sample all weights estimate

In [24]:
HessianNN_mnist = get_Hessian_NN(mnist_model, mnist_train_loader, prec0=30, device='cuda', verbose=True)

parameter size:  torch.Size([32, 1, 5, 5])
parameter size:  torch.Size([32])
parameter size:  torch.Size([64, 32, 5, 5])
parameter size:  torch.Size([64])
parameter size:  torch.Size([256, 1024])
parameter size:  torch.Size([256])
parameter size:  torch.Size([10, 256])
parameter size:  torch.Size([10])
Batch: 0/1875
Batch: 20/1875
Batch: 40/1875
Batch: 60/1875
Batch: 80/1875
Batch: 100/1875
Batch: 120/1875
Batch: 140/1875
Batch: 160/1875
Batch: 180/1875
Batch: 200/1875
Batch: 220/1875
Batch: 240/1875
Batch: 260/1875
Batch: 280/1875
Batch: 300/1875
Batch: 320/1875
Batch: 340/1875
Batch: 360/1875
Batch: 380/1875
Batch: 400/1875
Batch: 420/1875
Batch: 440/1875
Batch: 460/1875
Batch: 480/1875
Batch: 500/1875
Batch: 520/1875
Batch: 540/1875
Batch: 560/1875
Batch: 580/1875
Batch: 600/1875
Batch: 620/1875
Batch: 640/1875
Batch: 660/1875
Batch: 680/1875
Batch: 700/1875
Batch: 720/1875
Batch: 740/1875
Batch: 760/1875
Batch: 780/1875
Batch: 800/1875
Batch: 820/1875
Batch: 840/1875
Batch: 860/187

In [25]:
mnist_param_dist = get_param_dists(mnist_model, HessianNN_mnist)

mu size:  torch.Size([800])
Sigma size:  torch.Size([800])
mu size:  torch.Size([32])
Sigma size:  torch.Size([32])
mu size:  torch.Size([51200])
Sigma size:  torch.Size([51200])
mu size:  torch.Size([64])
Sigma size:  torch.Size([64])
mu size:  torch.Size([262144])
Sigma size:  torch.Size([262144])
mu size:  torch.Size([256])
Sigma size:  torch.Size([256])
mu size:  torch.Size([2560])
Sigma size:  torch.Size([2560])
mu size:  torch.Size([10])
Sigma size:  torch.Size([10])


In [26]:
# predict with different samples
def get_stacked_results(param_dists, data_loader, network, num_samples=100, verbose=False, timing=False):

    results = []
    if timing:
        time_sum = 0

    test_model = network(num_classes=10).cuda()
    test_model.eval()
    #new_state_dict = fmnist_model.state_dict()
    new_state_dict = dict()
    for s in range(num_samples):
        #torch.manual_seed(s)
        t0 = time.time()
        for idx, (p_name, param) in enumerate(test_model.named_parameters()):
            ps = param.size()
            new_weights = param_dists[idx].sample().view(ps)
            new_state_dict[p_name] = new_weights

        #print(new_state_dict.keys())
        test_model.load_state_dict(new_state_dict)
        
        inter_results = []
        for batch_idx, (x, y) in enumerate(data_loader):
            x = x.cuda()
            y_pred_logits = test_model(x).detach()
            #print(y_pred_logits)
            y_pred = F.softmax(y_pred_logits, dim=1).detach()
            inter_results.append(y_pred)
            if verbose and batch_idx % 100 == 0:
                print("s: {}; batch_idx: {}".format(s, batch_idx))
                
        results.append(torch.cat(inter_results, dim=0))
        t1 = time.time()
        if timing:
            time_sum += t1-t0
    
    stacked_results = torch.stack(results)
    print("time sum: ", time_sum)
    return(stacked_results)

In [27]:
stacked_results_mnist = get_stacked_results(param_dists=mnist_param_dist, data_loader=mnist_test_loader\
                                            ,network=LPADirNN, num_samples=100, verbose=False, timing=True)

time sum:  75.36639976501465


In [28]:
stacked_results_fmnist = get_stacked_results(param_dists=mnist_param_dist, data_loader=FMNIST_test_loader\
                                            ,network=LPADirNN, num_samples=100, verbose=False, timing=True)

time sum:  76.33020257949829


In [29]:
stacked_results_notmnist = get_stacked_results(param_dists=mnist_param_dist, data_loader=not_mnist_test_loader\
                                            ,network=LPADirNN, num_samples=100, verbose=False, timing=True)

time sum:  42.24465465545654


In [30]:
stacked_results_kmnist = get_stacked_results(param_dists=mnist_param_dist, data_loader=KMNIST_test_loader\
                                            ,network=LPADirNN, num_samples=100, verbose=False, timing=True)

time sum:  75.95980525016785


In [31]:
results_mnist = torch.mean(stacked_results_mnist, dim=0).cpu().numpy()
results_fmnist = torch.mean(stacked_results_fmnist, dim=0).cpu().numpy()
results_notmnist = torch.mean(stacked_results_notmnist, dim=0).cpu().numpy()
results_kmnist = torch.mean(stacked_results_kmnist, dim=0).cpu().numpy()

In [32]:
acc_in_S, prob_correct_in_S, ent_in_S, MMC_in_S = get_in_dist_values(results_mnist, targets)
acc_out_FMNIST_S, prob_correct_out_FMNIST_S, ent_out_FMNIST_S, MMC_out_FMNIST_S, auroc_out_FMNIST_S = get_out_dist_values(results_mnist, results_fmnist, targets_FMNIST)
acc_out_notMNIST_S, prob_correct_out_notMNIST_S, ent_out_notMNIST_S, MMC_out_notMNIST_S, auroc_out_notMNIST_S = get_out_dist_values(results_mnist, results_notmnist, targets_notMNIST)
acc_out_KMNIST_S, prob_correct_out_KMNIST_S, ent_out_KMNIST_S, MMC_out_KMNIST_S, auroc_out_KMNIST_S = get_out_dist_values(results_mnist, results_kmnist, targets_KMNIST)

In [33]:
print_in_dist_values(acc_in_S, prob_correct_in_S, ent_in_S, MMC_in_S, 'mnist', 'Sampling full')
print_out_dist_values(acc_out_FMNIST_S, prob_correct_out_FMNIST_S, ent_out_FMNIST_S, MMC_out_FMNIST_S, auroc_out_FMNIST_S, 'FMNIST', 'Sampling full')
print_out_dist_values(acc_out_notMNIST_S, prob_correct_out_notMNIST_S, ent_out_notMNIST_S, MMC_out_notMNIST_S, auroc_out_notMNIST_S, 'notMNIST', 'Sampling full')
print_out_dist_values(acc_out_KMNIST_S, prob_correct_out_KMNIST_S, ent_out_KMNIST_S, MMC_out_KMNIST_S, auroc_out_KMNIST_S, 'KMNIST', 'Sampling full')

[In, Sampling full, mnist] Accuracy: 0.989; average entropy: 0.071;     MMC: 0.979; Prob @ correct: 0.100
[Out-Sampling full, KFAC, FMNIST] Accuracy: 0.090; Average entropy: 1.486;    MMC: 0.473; AUROC: 0.991; Prob @ correct: 0.100
[Out-Sampling full, KFAC, notMNIST] Accuracy: 0.124; Average entropy: 0.978;    MMC: 0.647; AUROC: 0.954; Prob @ correct: 0.100
[Out-Sampling full, KFAC, KMNIST] Accuracy: 0.086; Average entropy: 0.997;    MMC: 0.633; AUROC: 0.966; Prob @ correct: 0.100


In [36]:
#Diag Sampling with 100 samples when sampling all weights
import numpy as np
#seeds are 123,124,125,126,127
time_sampling_all_in = [77.30279994010925, 75.88909602165222, 77.45202136039734, 75.38714742660522, 75.36639976501465]
time_sampling_all_out_fmnist = [75.75277590751648, 76.20959711074829, 76.57610511779785, 76.29606461524963, 76.33020257949829]
time_sampling_all_out_notmnist = [42.952558755874634, 42.86951684951782, 42.28128266334534, 42.60728096961975, 42.24465465545654]
time_sampling_all_out_kmnist = [76.0160961151123, 75.4875876903534, 76.32327890396118, 76.08931422233582, 75.95980525016785]

acc_in = [0.989, 0.989, 0.990, 0.989, 0.989]
mmc_in = [0.979, 0.980, 0.980, 0.979, 0.979]
mmc_out_fmnist = [0.463, 0.465, 0.461, 0.465, 0.473]
mmc_out_notmnist = [0.644, 0.644, 0.645, 0.645, 0.647]
mmc_out_kmnist = [0.632, 0.631, 0.635, 0.631, 0.633]

auroc_out_fmnist = [0.992, 0.992, 0.992, 0.991, 0.991]
auroc_out_notmnist = [0.956, 0.955, 0.955, 0.954, 0.954]
auroc_out_kmnist = [0.968, 0.968, 0.966, 0.968, 0.966]

print("Sampling time in: {:.03f} with std {:.03f}".format(np.mean(time_sampling_all_in), np.std(time_sampling_all_in)))
print("Sampling time out fmnist: {:.03f} with std {:.03f}".format(np.mean(time_sampling_all_out_fmnist), np.std(time_sampling_all_out_fmnist)))
print("Sampling time out notmnist: {:.03f} with std {:.03f}".format(np.mean(time_sampling_all_out_notmnist), np.std(time_sampling_all_out_notmnist)))
print("Sampling time out kmnist: {:.03f} with std {:.03f}".format(np.mean(time_sampling_all_out_kmnist), np.std(time_sampling_all_out_kmnist)))

print("accuracy: {:.03f} with std {:.03f}".format(np.mean(acc_in), np.std(acc_in)))

print("MMC in: {:.03f} with std {:.03f}".format(np.mean(mmc_in), np.std(mmc_in)))
print("MMC out fmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_fmnist), np.std(mmc_out_fmnist)))
print("MMC out notmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_notmnist), np.std(mmc_out_notmnist)))
print("MMC out kmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_kmnist), np.std(mmc_out_kmnist)))

print("AUROC out fmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_fmnist), np.std(auroc_out_fmnist)))
print("AUROC out notmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_notmnist), np.std(auroc_out_notmnist)))
print("AUROC out kmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_kmnist), np.std(auroc_out_kmnist)))

Sampling time in: 76.279 with std 0.917
Sampling time out fmnist: 76.233 with std 0.269
Sampling time out notmnist: 42.591 with std 0.291
Sampling time out kmnist: 75.975 with std 0.273
accuracy: 0.989 with std 0.000
MMC in: 0.979 with std 0.000
MMC out fmnist: 0.465 with std 0.004
MMC out notmnist: 0.645 with std 0.001
MMC out kmnist: 0.632 with std 0.001
AUROC out fmnist: 0.992 with std 0.000
AUROC out notmnist: 0.955 with std 0.001
AUROC out kmnist: 0.967 with std 0.001


In [35]:
#Diag Sampling with 1000 samples when sampling all weights
import numpy as np
#seeds are 123,124,125,126,127
time_sampling_all_in = [1175.2953972816467, 1188.364794254303, 1121.3945801258087, 1164.1343219280243, 1006.2414374351501]
time_sampling_all_out_fmnist = [1170.4062821865082, 1176.4168524742126, 1094.737497329712, 1143.832529783249, 999.0646016597748]
time_sampling_all_out_notmnist = [478.3686921596527, 473.6282377243042, 452.92081594467163, 429.9301836490631, 402.24077248573303]
time_sampling_all_out_kmnist = [1162.3914835453033, 1155.9036095142365, 1123.0117349624634, 1072.0317740440369, 990.6405141353607]

acc_in = [0.992, 0.992, 0.992, 0.992, 0.992]
mmc_in = [0.989, 0.989, 0.989, 0.989, 0.989]
mmc_out_fmnist = [0.540, 0.540, 0.540, 0.539, 0.539]
mmc_out_notmnist = [0.701, 0.701, 0.701, 0.702, 0.702]
mmc_out_kmnist = [0.672, 0.672, 0.672, 0.672, 0.672]

auroc_out_fmnist = [0.992, 0.992, 0.992, 0.992, 0.992]
auroc_out_notmnist = [0.960, 0.961, 0.961, 0.961, 0.960]
auroc_out_kmnist = [0.978, 0.978, 0.978, 0.978, 0.978]

print("Sampling time in: {:.03f} with std {:.03f}".format(np.mean(time_sampling_all_in), np.std(time_sampling_all_in)))
print("Sampling time out fmnist: {:.03f} with std {:.03f}".format(np.mean(time_sampling_all_out_fmnist), np.std(time_sampling_all_out_fmnist)))
print("Sampling time out notmnist: {:.03f} with std {:.03f}".format(np.mean(time_sampling_all_out_notmnist), np.std(time_sampling_all_out_notmnist)))
print("Sampling time out kmnist: {:.03f} with std {:.03f}".format(np.mean(time_sampling_all_out_kmnist), np.std(time_sampling_all_out_kmnist)))

print("accuracy: {:.03f} with std {:.03f}".format(np.mean(acc_in), np.std(acc_in)))

print("MMC in: {:.03f} with std {:.03f}".format(np.mean(mmc_in), np.std(mmc_in)))
print("MMC out fmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_fmnist), np.std(mmc_out_fmnist)))
print("MMC out notmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_notmnist), np.std(mmc_out_notmnist)))
print("MMC out kmnist: {:.03f} with std {:.03f}".format(np.mean(mmc_out_kmnist), np.std(mmc_out_kmnist)))

print("AUROC out fmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_fmnist), np.std(auroc_out_fmnist)))
print("AUROC out notmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_notmnist), np.std(auroc_out_notmnist)))
print("AUROC out kmnist: {:.03f} with std {:.03f}".format(np.mean(auroc_out_kmnist), np.std(auroc_out_kmnist)))

Sampling time in: 1131.086 with std 66.344
Sampling time out fmnist: 1116.892 with std 65.585
Sampling time out notmnist: 447.418 with std 28.362
Sampling time out kmnist: 1100.796 with std 63.696
accuracy: 0.992 with std 0.000
MMC in: 0.989 with std 0.000
MMC out fmnist: 0.540 with std 0.000
MMC out notmnist: 0.701 with std 0.000
MMC out kmnist: 0.672 with std 0.000
AUROC out fmnist: 0.992 with std 0.000
AUROC out notmnist: 0.961 with std 0.000
AUROC out kmnist: 0.978 with std 0.000
